In [1]:
import os
import zipfile

base_dir = "C:/Users/USER/Desktop/Reco/datasets"
image_dir = os.path.join(base_dir, "images")
mask_dir = os.path.join(base_dir, "masks")

In [2]:
# ===============================================================================
# 📋 STEP 0: 기본 설정 및 클래스 정의
# ===============================================================================

# 필수 임포트
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.optim import AdamW
from torch.optim.lr_scheduler import CosineAnnealingLR, ReduceLROnPlateau
from torch.amp import autocast, GradScaler
from transformers import AutoImageProcessor, AutoModelForSemanticSegmentation

import cv2
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import os
import re
import json
import random
import shutil
import zipfile
from tqdm import tqdm
from collections import Counter
import albumentations as A
from itertools import cycle

# 클래스 정의
class_names = [
    "background", "can", "glass",
    "paper", "plastic", "styrofoam", "vinyl"
]
label2id = {name: i for i, name in enumerate(class_names)}
id2label = {i: name for name, i in label2id.items()}
num_classes = len(class_names)

# 🔧 수정: 색상 매핑을 리스트로 통일
class_colors_bright = [
    (0, 0, 0),        # background - 검은색
    (0, 255, 255),    # can - 밝은 청록색 (Cyan)
    (255, 255, 0),    # glass - 밝은 노란색
    (128, 255, 0),    # paper - 연두색
    (255, 0, 0),      # plastic - 밝은 빨간색
    (0, 128, 255),    # styrofoam - 밝은 파란색
    (255, 0, 128)     # vinyl - 밝은 분홍색
]

# 디바이스 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"🔧 사용 디바이스: {device}")

c:\Users\USER\anaconda3\envs\cvl\lib\site-packages\transformers\utils\generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(


🔧 사용 디바이스: cuda


In [3]:
# ===========================================
# 🔍 STEP 1: 클래스별 픽셀 수 계산 (전처리)
# ===========================================

def preprocess_datasets():
    """datasets 폴더의 images와 masks를 전처리하고 클래스별 픽셀 수 계산"""
    print("\n" + "=" * 60)
    print("📊 STEP 1: datasets 폴더 데이터 전처리 시작")

    def get_base_name(filename):
        return os.path.splitext(filename)[0]

    def find_matching_files():
        image_list = [f for f in os.listdir(image_dir) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
        mask_list = [f for f in os.listdir(mask_dir) if f.lower().endswith('.png')]
        image_dict = {get_base_name(f): f for f in image_list}
        mask_dict = {get_base_name(f): f for f in mask_list}

        return [
            {'base_name': base_name, 'image_file': image_dict[base_name], 'mask_file': mask_dict[base_name]}
            for base_name in image_dict if base_name in mask_dict
        ]

    def remove_mask_noise(mask_array, min_area_ratio=0.003, min_pixels=25):
        cleaned_mask = mask_array.copy()
        total_pixels = mask_array.size

        for class_id in np.unique(mask_array):
            if class_id == 0:
                continue
            class_mask = (mask_array == class_id).astype(np.uint8)
            if class_mask.sum() < min_pixels:
                cleaned_mask[class_mask == 1] = 0
                continue

            num_labels, labels = cv2.connectedComponents(class_mask)
            components = sorted([
                (label_id, (labels == label_id).sum()) for label_id in range(1, num_labels)
            ], key=lambda x: x[1], reverse=True)

            for idx, (label_id, size) in enumerate(components):
                if idx < 3 or size / total_pixels >= min_area_ratio or size >= min_pixels:
                    continue
                cleaned_mask[labels == label_id] = 0

        return cleaned_mask

    final_data_list = []
    pixel_counter = Counter()
    matched_pairs = find_matching_files()

    for pair in tqdm(matched_pairs, desc="데이터 전처리"):
        img_path = os.path.join(image_dir, pair['image_file'])
        mask_path = os.path.join(mask_dir, pair['mask_file'])

        try:
            mask_array = np.array(Image.open(mask_path))
        except:
            continue

        remapped_mask = mask_array.copy()
        for class_id in np.unique(mask_array):
            if class_id == 0:
                continue
            if class_id in label2id.values():
                pixel_counter[class_id] += (mask_array == class_id).sum()
            else:
                remapped_mask[mask_array == class_id] = 0

        cleaned_mask = remove_mask_noise(remapped_mask)

        valid_classes = [cid for cid in np.unique(cleaned_mask) if cid != 0 and (cleaned_mask == cid).sum() >= 25]
        if not valid_classes:
            continue

        Image.fromarray(cleaned_mask.astype(np.uint8)).save(mask_path)

        final_data_list.append({
            "image": img_path,
            "label": mask_path,
            "class_ids": sorted(valid_classes),
            "base_name": pair['base_name']
        })

    return final_data_list, pixel_counter

In [4]:
# 함수 호출
final_data_list, pixel_counter = preprocess_datasets()

# 함수 밖에서 클래스별 픽셀 수 출력
print(f"\n📊 클래스별 픽셀 수:")
for class_id, count in sorted(pixel_counter.items()):
    class_name = id2label.get(class_id, f"class_{class_id}")
    print(f"   {class_name}: {count:,} pixels")


📊 STEP 1: datasets 폴더 데이터 전처리 시작


데이터 전처리: 100%|██████████| 587/587 [00:04<00:00, 136.05it/s]


📊 클래스별 픽셀 수:
   can: 3,891,709 pixels
   glass: 5,168,520 pixels
   paper: 4,241,993 pixels
   plastic: 8,331,825 pixels
   styrofoam: 6,468,212 pixels
   vinyl: 15,364,380 pixels


In [5]:
# ====================================================
# 🔧 STEP 2: 부족 클래스 중심 증강 (can, glass, paper)
# ====================================================

def create_smart_augmentation(is_train=True, input_size=512):
    """Rotate + Flip + Brightness/Contrast만 사용 (확률 높임)"""
    if is_train:
        return A.Compose([
            A.Resize(input_size, input_size),
            A.HorizontalFlip(p=0.7),
            A.Rotate(limit=10, p=0.7),
            A.RandomBrightnessContrast(
                brightness_limit=0.2,
                contrast_limit=0.2,
                p=0.7
            ),
        ], additional_targets={'mask': 'mask'})
    else:
        return A.Compose([
            A.Resize(input_size, input_size)
        ], additional_targets={'mask': 'mask'})

def advanced_class_specific_augmentation(final_data_list):
    print("\n" + "=" * 60)
    print("🔧 STEP 2: 클래스별 개별 증강 (can, glass, paper만)\n")

    target_class_names = ['can', 'glass', 'paper']
    target_classes = [label2id[name] for name in target_class_names]

    # 현재 픽셀 분포
    current_pixel_counts = {cid: 0 for cid in target_classes}
    class_to_images = {cid: [] for cid in target_classes}

    for rec in final_data_list:
        if "_aug" in os.path.basename(rec["image"]):
            continue
        mask = np.array(Image.open(rec["label"]).convert("L"))
        for cid in target_classes:
            pixel_count = (mask == cid).sum()
            if pixel_count > 0:
                current_pixel_counts[cid] += pixel_count
                class_to_images[cid].append({'record': rec, 'class_ratio': pixel_count / max((mask > 0).sum(), 1)})

    target_pixel = max(current_pixel_counts.values())
    print(f"🎯 목표 픽셀 수: {target_pixel:,}\n")

    aug_transform = create_smart_augmentation(is_train=True, input_size=512)
    augmented_list = []

    for cid in target_classes:
        need_pixel = target_pixel - current_pixel_counts[cid]
        if need_pixel <= 0:
            continue
        avg_pixel = current_pixel_counts[cid] / len(class_to_images[cid])
        total_aug = min(int(need_pixel / avg_pixel), len(class_to_images[cid]) * 3)

        print(f"{id2label[cid]} 증강 목표: {total_aug}개")
        images = class_to_images[cid]
        aug_count = 0

        for img_info in cycle(images):
            if aug_count >= total_aug:
                break

            rec = img_info['record']
            img = np.array(Image.open(rec["image"]).convert("RGB"))
            mask = np.array(Image.open(rec["label"]).convert("L"))

            for i in range(3):
                if aug_count >= total_aug:
                    break

                aug = aug_transform(image=img, mask=mask)
                aug_img, aug_mask = aug["image"], aug["mask"]

                if (aug_mask == cid).sum() < 30:
                    continue

                new_img = os.path.join(image_dir, f"{rec['base_name']}_cls{cid}_aug{aug_count+1}.jpg")
                new_mask = os.path.join(mask_dir, f"{rec['base_name']}_cls{cid}_aug{aug_count+1}.png")

                Image.fromarray(aug_img).save(new_img)
                Image.fromarray(aug_mask.astype(np.uint8)).save(new_mask)

                augmented_list.append({
                    "image": new_img,
                    "label": new_mask,
                    "class_ids": sorted(set(np.unique(aug_mask)) - {0}),
                    "base_name": rec['base_name']
                })

                aug_count += 1

    print(f"\n✅ 증강 완료: 총 {len(augmented_list)}개 생성")
    return final_data_list + augmented_list

def count_pixels_after_augmentation(final_data_list):
    """증강 후 전체 데이터에서 클래스별 픽셀 수 계산"""
    pixel_counter = Counter()

    for rec in tqdm(final_data_list, desc="증강 후 픽셀 수 계산"):
        mask_path = rec['label']

        try:
            mask = np.array(Image.open(mask_path))
        except:
            continue

        for class_id in np.unique(mask):
            if class_id == 0:
                continue
            pixel_counter[class_id] += (mask == class_id).sum()

    print("\n📊 증강 후 클래스별 픽셀 수:")
    for class_id, count in sorted(pixel_counter.items()):
        class_name = id2label.get(class_id, f"class_{class_id}")
        print(f"   {class_name}: {count:,} pixels")

    return pixel_counter

# STEP 1: 전처리 (원본 픽셀 수 확인)
final_data_list, pixel_counter = preprocess_datasets()

# STEP 2: 증강
augmented_data_list = advanced_class_specific_augmentation(final_data_list)

# STEP 3: 증강 후 픽셀 수 확인
final_pixel_counter = count_pixels_after_augmentation(augmented_data_list)
print(final_pixel_counter)

class ImprovedSegDataset(Dataset):
    """개선된 Segmentation Dataset"""

    def __init__(self, items, processor, is_train=True, input_size=512):
        self.items = items
        self.processor = processor
        self.input_size = input_size
        self.transform = create_smart_augmentation(is_train, input_size)
        self.max_class_id = max(label2id.values())

        # 🔧 유효한 아이템들만 필터링 (초기화 시 한 번만)
        self.valid_items = []
        print(f"📊 데이터셋 유효성 검사 중...")

        for i, item in enumerate(items):
            if os.path.exists(item['image']) and os.path.exists(item['label']):
                image_name = os.path.basename(item['image'])
                self.valid_items.append(item)
            else:
                print(f"⚠️ 파일 없음: {item}")

        print(f"✅ 유효한 데이터: {len(self.valid_items)}/{len(items)} 개")

    def __len__(self):
        return len(self.valid_items)

    def __getitem__(self, idx):
        # 🔧 인덱스 범위 체크
        if idx >= len(self.valid_items):
            idx = idx % len(self.valid_items)

        rec = self.valid_items[idx]

        try:
            # 🔧 이미지 로드 (BGR -> RGB 변환)
            image = cv2.imread(rec['image'])
            if image is None:
                raise ValueError(f"이미지 로드 실패: {rec['image']}")
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

            # 🔧 마스크 로드
            mask = cv2.imread(rec['label'], cv2.IMREAD_GRAYSCALE)
            if mask is None:
                raise ValueError(f"마스크 로드 실패: {rec['label']}")

            # 🔧 크기 체크 및 조정
            if image.shape[:2] != mask.shape[:2]:
                print(f"⚠️ 크기 불일치: {rec['image']} - img:{image.shape[:2]} vs mask:{mask.shape[:2]}")
                # 더 작은 크기로 통일
                h, w = min(image.shape[0], mask.shape[0]), min(image.shape[1], mask.shape[1])
                image = cv2.resize(image, (w, h), interpolation=cv2.INTER_AREA)
                mask = cv2.resize(mask, (w, h), interpolation=cv2.INTER_NEAREST)

            # 🔧 마스크 값 클리핑 (중요!)
            mask = np.clip(mask, 0, self.max_class_id)

            # 🔧 마스크 값 검사 (디버깅용)
            unique_vals = np.unique(mask)
            if len(unique_vals) > len(label2id):
                print(f"⚠️ 예상치 못한 마스크 값: {unique_vals} in {rec['label']}")

            # 🔧 Augmentation 적용
            if self.transform:
                try:
                    aug = self.transform(image=image, mask=mask)
                    image, mask = aug['image'], aug['mask']
                except Exception as e:
                    print(f"⚠️ Augmentation 오류: {e}")
                    # Augmentation 실패 시 원본 사용

            # 🔧 Processor 적용
            try:
                proc = self.processor(images=image, return_tensors="pt")
                pixel_values = proc["pixel_values"].squeeze(0)
            except Exception as e:
                print(f"⚠️ Processor 오류: {e}")
                # 직접 텐서 변환
                image_tensor = torch.tensor(image.transpose(2, 0, 1), dtype=torch.float32) / 255.0
                pixel_values = image_tensor

            # 🔧 라벨 텐서 변환
            labels = torch.tensor(mask, dtype=torch.long)

            # 🔧 파일 정보 추가 (파일명 + 원본 경로)
            filename = os.path.basename(rec['image'])
            original_image_path = rec['image']  # 전체 원본 이미지 경로

            return {
                "pixel_values": pixel_values,
                "labels": labels,
                "filename": filename,
                "original_image_path": rec['image']
            }

        except Exception as e:
            print(f"⚠️ 데이터 로딩 오류: {rec['image']} - {str(e)}")

            # 🔧 재귀 호출 시 무한루프 방지
            if idx == 0:
                # 첫 번째 아이템도 실패하면 더미 데이터 반환
                return self._get_dummy_data()
            else:
                # 다른 인덱스 시도
                return self.__getitem__(0)

    def _get_dummy_data(self):
        """오류 발생 시 더미 데이터 반환"""
        print("🔧 더미 데이터 생성 중...")

        # 더미 이미지 (512x512 RGB)
        dummy_image = torch.zeros(3, self.input_size, self.input_size, dtype=torch.float32)

        # 더미 마스크 (512x512, 모든 값이 0)
        dummy_mask = torch.zeros(self.input_size, self.input_size, dtype=torch.long)

        return {
            "pixel_values": dummy_image,
            "labels": dummy_mask,
            "filename": "dummy.jpg"
        }

    def get_class_distribution(self):
        """클래스 분포 확인용 메서드"""
        class_counts = {class_name: 0 for class_name in class_names}

        print("📊 클래스 분포 분석 중...")
        for item in tqdm(self.valid_items[:100]):  # 샘플 100개만 체크
            try:
                mask = cv2.imread(item['label'], cv2.IMREAD_GRAYSCALE)
                if mask is not None:
                    unique_vals, counts = np.unique(mask, return_counts=True)
                    for val, count in zip(unique_vals, counts):
                        if val < len(class_names):
                            class_counts[class_names[val]] += count
            except Exception as e:
                continue

        return class_counts


# 🔧 데이터셋 생성 시 유틸리티 함수
def create_dataset_with_validation(items, processor, is_train=True, input_size=512):
    """
    데이터셋 생성 전에 데이터 유효성 검사
    """
    print(f"🔍 데이터 유효성 검사 시작...")

    valid_items = []
    invalid_items = []

    for item in items:
        # 파일 존재 여부 확인
        if not os.path.exists(item['image']):
            invalid_items.append(f"이미지 없음: {item['image']}")
            continue
        if not os.path.exists(item['label']):
            invalid_items.append(f"라벨 없음: {item['label']}")
            continue

        # 파일 크기 체크
        try:
            img_size = os.path.getsize(item['image'])
            mask_size = os.path.getsize(item['label'])
            if img_size == 0 or mask_size == 0:
                invalid_items.append(f"빈 파일: {item}")
                continue
        except:
            invalid_items.append(f"파일 접근 오류: {item}")
            continue

        valid_items.append(item)

    print(f"✅ 유효한 데이터: {len(valid_items)}")
    print(f"❌ 무효한 데이터: {len(invalid_items)}")

    if invalid_items:
        print("⚠️ 무효한 데이터 예시:")
        for item in invalid_items[:5]:  # 처음 5개만 출력
            print(f"   - {item}")

    # 데이터셋 생성
    dataset = ImprovedSegDataset(valid_items, processor, is_train, input_size)

    return dataset


📊 STEP 1: datasets 폴더 데이터 전처리 시작


데이터 전처리: 100%|██████████| 587/587 [00:04<00:00, 142.11it/s]



🔧 STEP 2: 클래스별 개별 증강 (can, glass, paper만)

🎯 목표 픽셀 수: 3,639,054

glass 증강 목표: 39개
paper 증강 목표: 16개

✅ 증강 완료: 총 55개 생성


증강 후 픽셀 수 계산: 100%|██████████| 642/642 [00:01<00:00, 588.25it/s]


📊 증강 후 클래스별 픽셀 수:
   can: 4,144,430 pixels
   glass: 7,927,088 pixels
   paper: 5,329,988 pixels
   plastic: 9,227,584 pixels
   styrofoam: 6,468,212 pixels
   vinyl: 16,547,384 pixels
Counter({np.uint8(6): np.int64(16547384), np.uint8(4): np.int64(9227584), np.uint8(2): np.int64(7927088), np.uint8(5): np.int64(6468212), np.uint8(3): np.int64(5329988), np.uint8(1): np.int64(4144430)})


In [6]:
# ===============================================================================
# 🚀 STEP 3: 개선된 Loss 함수 및 학습 시스템 (Enhanced Version)
# ===============================================================================
# 🔧 기존 코드 99% 유지 + 3가지 작은 수정만

import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import cv2
from collections import Counter
from PIL import Image
import os
import math
from torch.optim import AdamW
from torch.optim.lr_scheduler import ReduceLROnPlateau

# 기존 코드 그대로 유지 (CombinedBoundaryLoss, EnhancedCombinedLoss, calculate_improved_weights, 등등...)
class CombinedBoundaryLoss(nn.Module):
    def __init__(self,
                 class_weights=None,
                 dice_weight=0.5,
                 ce_weight=0.3,
                 boundary_weight=0.2,
                 smooth=1e-7):
        super().__init__()
        self.class_weights = class_weights
        self.dice_weight = dice_weight
        self.ce_weight = ce_weight
        self.boundary_weight = boundary_weight
        self.smooth = smooth

        # Sobel 필터(경계 검출용) 행렬을 미리 CPU 텐서로 만들어 둡니다.
        sobel_x = torch.tensor([[-1, 0, 1],
                                [-2, 0, 2],
                                [-1, 0, 1]],
                               dtype=torch.float32).unsqueeze(0).unsqueeze(0)  # shape (1,1,3,3)
        sobel_y = torch.tensor([[-1, -2, -1],
                                [ 0,  0,  0],
                                [ 1,  2,  1]],
                               dtype=torch.float32).unsqueeze(0).unsqueeze(0)  # shape (1,1,3,3)

        # register_buffer 하면 .to(device) 호출 시 이 버퍼도 자동으로 이동합니다.
        self.register_buffer('sobel_x', sobel_x)
        self.register_buffer('sobel_y', sobel_y)

    def forward(self, logits: torch.Tensor, targets: torch.Tensor) -> torch.Tensor:
        """
        logits: (B, num_classes, H, W), torch.cuda.FloatTensor (모델 출력)
        targets: (B, H, W), torch.cuda.LongTensor (GT 클래스)
        """

        # -------------------------------------------
        # 1) CrossEntropy 손실 (CE)
        # -------------------------------------------
        ce_loss = F.cross_entropy(logits, targets, weight=self.class_weights)

        # -------------------------------------------
        # 2) Dice Loss (기존 방식 + 안정성 개선)
        # -------------------------------------------
        probs = F.softmax(logits, dim=1)  # (B, num_classes, H, W)
        dice_losses = []
        num_classes = logits.shape[1]
        for cls in range(1, num_classes):  # 배경(0) 제외
            t_cls = (targets == cls).float()  # (B, H, W)
            p_cls = probs[:, cls]             # (B, H, W)
            inter = (p_cls * t_cls).sum(dim=[1,2])   # 배치별 합
            union = p_cls.sum(dim=[1,2]) + t_cls.sum(dim=[1,2])
            dice_score = ((2 * inter + self.smooth) / (union + self.smooth))
            dice_losses.append(1 - dice_score)  # 1 - dice → loss

        if dice_losses:
            # dice_losses: list of (B,) → stack → (B, num_fg_classes) → mean → scalar
            dice_loss = torch.stack(dice_losses, dim=1).mean()
        else:
            dice_loss = torch.tensor(0.0, device=logits.device)

        # -------------------------------------------
        # 3) Boundary Loss (Sobel edge 기반 L1) + 개선사항
        # -------------------------------------------
        # pred_mask(0~C-1 정수) 추출
        pred_mask = torch.argmax(probs, dim=1, keepdim=True).float()  # (B, 1, H, W)
        gt_mask = targets.unsqueeze(1).float()                         # (B, 1, H, W)

        # Sobel 커널을 logits.device로 옮겨서 사용 (register_buffer로 자동 이동)
        # (a) pred 경계
        gx_pred = F.conv2d(pred_mask, self.sobel_x, padding=1)
        gy_pred = F.conv2d(pred_mask, self.sobel_y, padding=1)
        edge_pred = torch.sqrt(gx_pred ** 2 + gy_pred ** 2 + 1e-8)  # 수치 안정성

        # (b) GT 경계
        gx_gt = F.conv2d(gt_mask, self.sobel_x, padding=1)
        gy_gt = F.conv2d(gt_mask, self.sobel_y, padding=1)
        edge_gt = torch.sqrt(gx_gt ** 2 + gy_gt ** 2 + 1e-8)

        # (c) 경계 픽셀 L1 차이 (경계 영역에만 집중)
        edge_mask = (edge_gt > 0.1).float()  # 실제 경계가 있는 곳만
        if edge_mask.sum() > 0:
            boundary_loss = F.l1_loss(edge_pred * edge_mask, edge_gt * edge_mask)
        else:
            boundary_loss = torch.tensor(0.0, device=logits.device)

        # -------------------------------------------
        # 4) 최종 가중합
        # -------------------------------------------
        total_loss = (
            self.ce_weight * ce_loss +
            self.dice_weight * dice_loss +
            self.boundary_weight * boundary_loss
        )
        return total_loss

# 🆕 추가: Focal Loss 조합 버전
class EnhancedCombinedLoss(nn.Module):
    """
    CombinedBoundaryLoss + Focal Loss의 조합
    클래스 불균형이 심할 때 더 효과적
    """
    def __init__(self,
                 class_weights=None,
                 dice_weight=0.4,
                 ce_weight=0.2,
                 focal_weight=0.2,
                 boundary_weight=0.2,
                 focal_alpha=1.0,
                 focal_gamma=2.0):
        super().__init__()
        self.boundary_loss = CombinedBoundaryLoss(
            class_weights=class_weights,
            dice_weight=dice_weight,
            ce_weight=ce_weight,
            boundary_weight=boundary_weight
        )
        self.focal_weight = focal_weight
        self.focal_alpha = focal_alpha
        self.focal_gamma = focal_gamma

    def focal_loss(self, pred, target):
        ce_loss = F.cross_entropy(pred, target, reduction='none')
        pt = torch.exp(-ce_loss)
        focal_loss = self.focal_alpha * (1 - pt) ** self.focal_gamma * ce_loss
        return focal_loss.mean()

    def forward(self, logits, targets):
        boundary_loss = self.boundary_loss(logits, targets)
        focal_loss = self.focal_loss(logits, targets)
        return boundary_loss + self.focal_weight * focal_loss

def calculate_improved_weights(data_list, device='cuda'):
    """클래스 개수 맞춘 개선된 가중치 계산 (sqrt 방식 고정)"""
    pixel_counter = Counter()

    for rec in data_list:
        try:
            mask = np.array(Image.open(rec["label"]).convert("L"))
            unique, counts = np.unique(mask, return_counts=True)
            for cls_id, count in zip(unique, counts):
                if 0 <= cls_id <= 6:  # ✅ 0~6만 사용
                    pixel_counter[cls_id] += count
        except:
            continue

    fg_pixels = {k: v for k, v in pixel_counter.items() if k > 0}
    total_fg = sum(fg_pixels.values())

    weights = np.ones(7)  # ✅ 총 7개 클래스
    weights[0] = 0.05  # 배경 가중치는 작게

    for cls_id in range(1, 7):
        if cls_id in fg_pixels:
            freq = fg_pixels[cls_id] / total_fg
            weights[cls_id] = np.sqrt(1.0 / (freq + 1e-6))  # sqrt 방식 고정

    # 정규화 (foreground 클래스만)
    weights[1:] = weights[1:] / weights[1:].sum() * 6

    print(f"\n📊 클래스 가중치 (sqrt 기준):")
    for i, w in enumerate(weights):
        class_name = id2label.get(i, f"class_{i}")
        print(f"  {class_name}: {w:.3f}")

    return torch.tensor(weights, dtype=torch.float32, device=device)

# 🔧 수정 1: TTA 부분만 개선 (기존 코드 99% 유지)
def gentle_predict(batch, model, input_size=512, num_classes=10,
                   confidence_threshold=None, use_multiscale=False, use_tta=False):
    """
    기존 기능 + 개선된 TTA (기존 코드 99% 유지)
    """
    imgs = batch["pixel_values"].to(device)

    # "original_image" 키가 있으면 CRF까지 수행
    do_crf = "original_image" in batch
    if do_crf:
        orig_images = batch["original_image"]

    with torch.no_grad():
        if use_multiscale:
            # Multi-scale testing (기존 코드 그대로)
            probs = multi_scale_predict(model, imgs)
        elif use_tta:
            # 🔧 개선된 TTA (4가지 변환으로 확장)
            tta_preds = []

            # 1) 원본
            outputs = model(pixel_values=imgs)
            logits = outputs.logits
            if logits.shape[-2:] != (input_size, input_size):
                logits = F.interpolate(logits, size=(input_size, input_size), mode="bilinear", align_corners=False)
            tta_preds.append(F.softmax(logits, dim=1))

            # 2) 좌우 반전
            imgs_h_flipped = torch.flip(imgs, dims=[3])
            outputs = model(pixel_values=imgs_h_flipped)
            logits = outputs.logits
            if logits.shape[-2:] != (input_size, input_size):
                logits = F.interpolate(logits, size=(input_size, input_size), mode="bilinear", align_corners=False)
            logits_h_flipped_back = torch.flip(logits, dims=[3])
            tta_preds.append(F.softmax(logits_h_flipped_back, dim=1))

            # 3) 🆕 상하 반전 추가
            imgs_v_flipped = torch.flip(imgs, dims=[2])
            outputs = model(pixel_values=imgs_v_flipped)
            logits = outputs.logits
            if logits.shape[-2:] != (input_size, input_size):
                logits = F.interpolate(logits, size=(input_size, input_size), mode="bilinear", align_corners=False)
            logits_v_flipped_back = torch.flip(logits, dims=[2])
            tta_preds.append(F.softmax(logits_v_flipped_back, dim=1))

            # 4) 🆕 상하좌우 반전 추가
            imgs_both_flipped = torch.flip(imgs, dims=[2, 3])
            outputs = model(pixel_values=imgs_both_flipped)
            logits = outputs.logits
            if logits.shape[-2:] != (input_size, input_size):
                logits = F.interpolate(logits, size=(input_size, input_size), mode="bilinear", align_corners=False)
            logits_both_flipped_back = torch.flip(logits, dims=[2, 3])
            tta_preds.append(F.softmax(logits_both_flipped_back, dim=1))

            # 평균 (4개 변환 결과)
            probs = torch.stack(tta_preds).mean(dim=0)
        else:
            # 기본 예측 (기존 코드 그대로)
            outputs = model(pixel_values=imgs)
            logits = outputs.logits
            if logits.shape[-2:] != (input_size, input_size):
                logits = F.interpolate(logits, size=(input_size, input_size), mode="bilinear", align_corners=False)
            probs = F.softmax(logits, dim=1)

        # 후처리 (기존 코드 완전히 그대로)
        filtered_pred_list = []
        for i in range(probs.shape[0]):
            single_probs = probs[i].cpu().numpy()
            pred_mask = np.argmax(single_probs, axis=0)

            # 1) 면적+confidence 기준 노이즈 제거 (adaptive threshold 사용)
            filtered1 = remove_tiny_noise_with_confidence(
                pred_mask,
                single_probs,
                min_area_ratio=0.003,
                conf_thresh=0.3,
                adaptive_thresh=True  # 🆕 추가
            )

            # 2) 형태학적 연산 적용
            filtered2 = refine_mask_morphology(filtered1, kernel_size=5)

            if do_crf:
                # 3) CRF 후처리
                img_np = (orig_images[i].cpu().numpy().transpose(1, 2, 0) * 255).astype(np.uint8)
                refined_mask = apply_postprocessing(img_np, single_probs)
                cleaned_pred = refined_mask
            else:
                cleaned_pred = filtered2

            filtered_pred_list.append(torch.tensor(cleaned_pred, device=device))

        pred = torch.stack(filtered_pred_list)

    return probs, pred

from torch.cuda.amp import autocast, GradScaler

# 🔧 수정 2: 함수 정의 순서 수정 (improved_training 함수 완성)
def improved_training(model, train_loader, val_loader, processor, class_weights_tensor,
                     max_epochs=400, patience=40, device='cuda',
                     use_enhanced_loss=False, use_advanced_scheduler=False):
    """
    기존 학습 함수 + 추가 옵션들 (완전한 함수 정의)
    """
    # 🔧 GPU 최적화 설정 추가
    torch.backends.cudnn.benchmark = True
    torch.backends.cuda.matmul.allow_tf32 = True
    torch.backends.cudnn.allow_tf32 = True

    model = model.to(device)
    # 🔧 Mixed Precision 스케일러 추가
    scaler = GradScaler()
    optimizer = AdamW(model.parameters(), lr=1e-4, weight_decay=1e-4, eps=1e-8)

    if use_advanced_scheduler:
        scheduler = create_advanced_scheduler(optimizer, max_epochs)
        scheduler_type = 'cosine'
    else:
        scheduler = ReduceLROnPlateau(optimizer, mode='max',
                                      factor=0.7, patience=12,
                                      threshold=0.005, min_lr=1e-6, verbose=True)
        scheduler_type = 'plateau'

    # Loss function 선택
    if use_enhanced_loss:
        criterion = EnhancedCombinedLoss(
            class_weights=class_weights_tensor,
            dice_weight=0.4,
            ce_weight=0.2,
            focal_weight=0.2,
            boundary_weight=0.2
        ).to(device)
        print("🔥 Using EnhancedCombinedLoss (with Focal Loss)")
    else:
        criterion = CombinedBoundaryLoss(
            class_weights=class_weights_tensor,
            dice_weight=0.5,
            ce_weight=0.3,
            boundary_weight=0.2
        ).to(device)
        print("🎯 Using CombinedBoundaryLoss")

    history = {'train_loss': [], 'val_loss': [], 'dice_scores': [], 'iou_scores': [], 'learning_rates': []}
    best_dice = 0.0
    early_stop_counter = 0

    for epoch in range(1, max_epochs + 1):
        # ----- Training -----
        model.train()
        train_losses = []
        for batch in train_loader:
            imgs = batch["pixel_values"].to(device)
            masks = batch["labels"].to(device)
            masks = torch.clamp(masks, 0, 9)
            masks = F.interpolate(
                masks.unsqueeze(1).float(),
                size=(512, 512),
                mode='nearest'
            ).squeeze(1).long()

            optimizer.zero_grad()
    
            # 🔧 Mixed Precision으로 감싸기
            with autocast():
                outputs = model(pixel_values=imgs)
                logits = outputs.logits
                logits = F.interpolate(logits, size=(512, 512), mode='bilinear')
                loss = criterion(logits, masks)
    
            # 🔧 Mixed Precision으로 backward
            scaler.scale(loss).backward()
            scaler.unscale_(optimizer)  # gradient clipping 전에 추가
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            scaler.step(optimizer)
            scaler.update()

            train_losses.append(loss.item())

        avg_train_loss = np.mean(train_losses)
        current_lr = optimizer.param_groups[0]['lr']

        # ----- Validation dice/iou -----
        val_dice, val_iou = evaluate_model_fairly(model, val_loader, use_strict=False)

        # ----- Validation loss -----
        model.eval()
        val_losses = []
        with torch.no_grad():
            for batch in val_loader:
                imgs = batch["pixel_values"].to(device)
                masks = batch["labels"].to(device)
                masks = torch.clamp(masks, 0, 9)
                masks = F.interpolate(
                    masks.unsqueeze(1).float(),
                    size=(512, 512),
                    mode='nearest'
                ).squeeze(1).long()

                # 🔧 Validation에도 autocast 적용
                with autocast():
                    outputs = model(pixel_values=imgs)
                    logits = outputs.logits
                    logits = F.interpolate(logits, size=(512, 512), mode='bilinear')
                    loss = criterion(logits, masks)
                val_losses.append(loss.item())

        avg_val_loss = np.mean(val_losses) if val_losses else float('inf')

        # Scheduler step
        if scheduler_type == 'plateau':
            scheduler.step(val_dice)
        else:
            scheduler.step()

        history['train_loss'].append(avg_train_loss)
        history['val_loss'].append(avg_val_loss)
        history['dice_scores'].append(val_dice)
        history['iou_scores'].append(val_iou)
        history['learning_rates'].append(current_lr)

        print(f"Epoch {epoch}/{max_epochs}  ▶  Train Loss: {avg_train_loss:.4f}  |  Val Loss: {avg_val_loss:.4f}  |  Dice: {val_dice:.4f}  |  IoU: {val_iou:.4f}  |  LR: {current_lr:.2e}")

        # ----- Best 모델 저장 & 
        # 
        # 
        #  Stopping -----
        print(f"🔍 현재 val_dice: {val_dice:.4f}, 현재 best_dice: {best_dice:.4f}")  # 🔧 디버깅 추가
        
        if val_dice > best_dice:
            print(f"🎉 새로운 Best 발견! {best_dice:.4f} → {val_dice:.4f}")  # 🔧 디버깅 추가
            best_dice = val_dice
            save_dir = "C:/Users/USER/Desktop/Reco/results/best_model"
            
            try:
                os.makedirs(save_dir, exist_ok=True)
                print(f"📁 폴더 생성 완료: {save_dir}")  # 🔧 디버깅 추가

                model.eval()  # 혹시 모르니 다시 한 번 평가 모드
                model.save_pretrained(save_dir)
                processor.save_pretrained(save_dir)

                # 🔍 저장 확인
                saved_files = os.listdir(save_dir)
                print(f"💾 저장된 파일들: {saved_files}")
                print(f"✅ New Best! Dice {val_dice:.4f} 저장 완료")

            except Exception as e:
                print(f"❌ 모델 저장 실패: {e}")  # 🔧 에러 확인
                
            early_stop_counter = 0
        else:
            print(f"⏭️ 개선 없음 (현재: {val_dice:.4f} ≤ 최고: {best_dice:.4f}") 
            print(f"Patience: {early_stop_counter}/{patience}")  # 🔧 디버깅 추가
            early_stop_counter += 1

        if early_stop_counter >= patience:
            print(f"\n🛑 Early stopping at epoch {epoch}")
            break

    return history, save_dir

# 🔧 기존 코드들 그대로 유지 (나머지 모든 함수들)
def remove_tiny_noise_with_confidence(pred_mask: np.ndarray,
                                      single_probs: np.ndarray,
                                      min_area_ratio: float = 0.003,
                                      conf_thresh: float = 0.3,
                                      adaptive_thresh: bool = True) -> np.ndarray:
    """기존 함수 + adaptive threshold 옵션 추가"""
    H, W = pred_mask.shape
    total_pixels = H * W
    filtered_mask = np.zeros_like(pred_mask)

    for cls_id in np.unique(pred_mask):
        if cls_id == 0:
            continue  # 배경은 건너뛴다

        class_mask = (pred_mask == cls_id).astype(np.uint8)
        num_labels, labels = cv2.connectedComponents(class_mask)

        # 🆕 Adaptive threshold: 클래스별 평균 confidence 기반
        if adaptive_thresh:
            class_confidences = single_probs[cls_id][class_mask == 1]
            if len(class_confidences) > 0:
                adaptive_conf_thresh = max(conf_thresh, np.percentile(class_confidences, 75))
            else:
                adaptive_conf_thresh = conf_thresh
        else:
            adaptive_conf_thresh = conf_thresh

        for label_id in range(1, num_labels):
            component_mask = (labels == label_id)
            area = component_mask.sum()
            area_ratio = area / total_pixels

            if area_ratio >= min_area_ratio:
                # 면적이 충분히 크면 무조건 보존
                filtered_mask[component_mask] = cls_id
            else:
                # 작은 덩어리인 경우, 내부 픽셀 확률을 확인
                comp_confidences = single_probs[cls_id][component_mask]
                max_conf = comp_confidences.max() if comp_confidences.size > 0 else 0.0

                if max_conf >= adaptive_conf_thresh:
                    # 작은 덩어리이지만 충분한 확신이 있으면 보존
                    filtered_mask[component_mask] = cls_id

    return filtered_mask

def multi_scale_predict(model, image, scales=[0.75, 1.0, 1.25], base_size=512):
    """Multi-scale testing으로 성능 향상"""
    device = next(model.parameters()).device
    model.eval()

    # 원본 크기 저장
    _, _, h_orig, w_orig = image.shape

    all_preds = []

    with torch.no_grad():
        for scale in scales:
            # 스케일에 따른 크기 계산
            h_scaled = int(h_orig * scale)
            w_scaled = int(w_orig * scale)

            # 32의 배수로 조정 (모델 요구사항)
            h_scaled = (h_scaled // 32) * 32
            w_scaled = (w_scaled // 32) * 32

            # 이미지 리사이즈
            image_scaled = F.interpolate(
                image,
                size=(h_scaled, w_scaled),
                mode='bilinear',
                align_corners=False
            )

            # 예측
            outputs = model(pixel_values=image_scaled)
            logits = outputs.logits

            # 원본 크기로 복원
            logits = F.interpolate(
                logits,
                size=(base_size, base_size),
                mode='bilinear',
                align_corners=False
            )

            # Softmax 적용
            probs = F.softmax(logits, dim=1)
            all_preds.append(probs)

    # 평균 앙상블
    final_probs = torch.stack(all_preds).mean(dim=0)
    return final_probs

def refine_mask_morphology(mask: np.ndarray, kernel_size: int = 5) -> np.ndarray:
    """Opening → Closing 형태학적 연산을 통해 경계를 매끄럽게 함"""
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (kernel_size, kernel_size))
    opened = cv2.morphologyEx(mask.astype(np.uint8), cv2.MORPH_OPEN, kernel)
    closed = cv2.morphologyEx(opened, cv2.MORPH_CLOSE, kernel)
    return closed
    
def apply_postprocessing(image_np: np.ndarray, prob_map_np: np.ndarray) -> np.ndarray:
    return np.argmax(prob_map_np, axis=0)

def evaluate_model_fairly(model, val_loader, use_strict=False):
    model.eval()
    all_preds = []
    all_targets = []
    with torch.no_grad():
        for batch in val_loader:
            # ⚠️ threshold 없는 호출
            _, preds = gentle_predict(batch, model, 512, 10)

            targets = batch["labels"].to(device)
            targets = F.interpolate(
                targets.unsqueeze(1).float(),
                size=(512, 512),
                mode="nearest"
            ).squeeze(1).long()

            all_preds.append(preds.cpu().numpy())
            all_targets.append(targets.cpu().numpy())

    if all_preds and all_targets:
        pred_flat = np.concatenate([p.flatten() for p in all_preds])
        target_flat = np.concatenate([t.flatten() for t in all_targets])
        dice, iou, _ = calculate_advanced_metrics(pred_flat, target_flat, 10)
        return dice, iou

    return 0.0, 0.0

def calculate_advanced_metrics(pred, target, num_classes):
    pred = np.array(pred).flatten()
    target = np.array(target).flatten()

    valid_mask = target != 0
    pred_valid = pred[valid_mask]
    target_valid = target[valid_mask]

    if len(target_valid) == 0:
        return 0.0, 0.0, np.zeros(num_classes)

    dice_scores = []
    iou_scores = []
    precision_scores = np.zeros(num_classes)

    for cls in range(1, num_classes):
        pred_cls = (pred_valid == cls)
        target_cls = (target_valid == cls)

        if target_cls.sum() == 0:
            continue

        intersection = (pred_cls & target_cls).sum()
        union = (pred_cls | target_cls).sum()

        if union > 0:
            iou = intersection / union
            dice = (2 * intersection) / (pred_cls.sum() + target_cls.sum())
            iou_scores.append(iou)
            dice_scores.append(dice)

        if pred_cls.sum() > 0:
            precision_scores[cls] = intersection / pred_cls.sum()

    return (np.mean(dice_scores) if dice_scores else 0.0,
            np.mean(iou_scores) if iou_scores else 0.0,
            precision_scores)

def create_advanced_scheduler(optimizer, num_epochs, warmup_epochs=5):
    """Warmup + Cosine Annealing with Restarts"""
    def lr_lambda(epoch):
        if epoch < warmup_epochs:
            # Linear warmup
            return epoch / warmup_epochs
        else:
            # Cosine annealing with restarts
            progress = (epoch - warmup_epochs) / (num_epochs - warmup_epochs)
            return 0.5 * (1 + math.cos(math.pi * progress))

    return torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda)

In [7]:
# ===============================================================================
# 📊 STEP 4: 결과 평가 및 저장 (수정된 버전)
# ===============================================================================

import os
import cv2
import numpy as np
import torch
import matplotlib.pyplot as plt
from tqdm import tqdm
from PIL import Image

# ──────────────────────────────────────────────────────────────────────────────
# 1) 마스크를 밝은 RGB 컬러로 변환
# ──────────────────────────────────────────────────────────────────────────────

def mask_to_color_rgb(mask: np.ndarray) -> np.ndarray:
    """🔧 RGB 기반 마스크 색상 변환 - 밝은 색상 사용"""
    # 🔍 디버깅 추가
    print(f"🔧 Mask unique values: {np.unique(mask)}")
    print(f"🔧 Available colors: {len(class_colors_bright) if 'class_colors_bright' in globals() else 'NOT DEFINED'}")

    color_mask = np.zeros((*mask.shape, 3), dtype=np.uint8)

    # 🔧 각 클래스별로 색상 적용 및 확인
    for cid in range(len(class_colors_bright)):
        if cid in mask:  # 실제로 마스크에 해당 클래스가 있는지 확인
            color = class_colors_bright[cid]
            pixel_count = np.sum(mask == cid)
            color_mask[mask == cid] = color
            print(f"🔧 Class {cid}: {pixel_count} pixels → RGB{color}")

    # 🔧 최종 결과 확인
    print(f"🔧 Color mask shape: {color_mask.shape}")
    print(f"🔧 Color mask range: {color_mask.min()} ~ {color_mask.max()}")
    if color_mask.max() > 0:
        print(f"🔧 Color mask 채널별: R={color_mask[:,:,0].mean():.1f}, G={color_mask[:,:,1].mean():.1f}, B={color_mask[:,:,2].mean():.1f}")

    return color_mask

# ──────────────────────────────────────────────────────────────────────────────
# 2) 읽기 쉬운 텍스트를 중앙 정렬해서 그린다
# ──────────────────────────────────────────────────────────────────────────────

def draw_readable_text(image, text, center_pos, font_scale=0.5, spacing_factor=1.3):
    """✨ 자간과 두께가 적절한 읽기 쉬운 텍스트"""
    font = cv2.FONT_HERSHEY_SIMPLEX
    thickness = 1  # 🔧 두께를 1로 고정 (얇게)

    total_width = 0
    char_infos = []

    for char in text:
        (char_w, char_h), baseline = cv2.getTextSize(char, font, font_scale, thickness)
        char_infos.append((char, char_w, char_h, baseline))
        total_width += int(char_w * spacing_factor)

    start_x = center_pos[0] - total_width // 2
    y = center_pos[1]
    current_x = start_x

    for i, (char, char_w, char_h, baseline) in enumerate(char_infos):
        cv2.putText(image, char, (current_x, y), font, font_scale, (255, 255, 255), thickness, cv2.LINE_AA)
        current_x += int(char_w * spacing_factor)

    text_height = max([info[2] for info in char_infos]) if char_infos else 0
    return total_width, text_height

# ──────────────────────────────────────────────────────────────────────────────
# 3) 세그먼트 중심(centroid)에 라벨을 읽기 쉽게 붙여 준다
# ──────────────────────────────────────────────────────────────────────────────

def add_readable_center_labels(image_rgb: np.ndarray,
                               pred_mask: np.ndarray,
                               class_names: dict,
                               label_scale: float = 0.6) -> np.ndarray:
    """✨ 읽기 쉬운 중앙 라벨링 (분리된 각 컴포넌트 중심에 텍스트 추가)"""
    result = image_rgb.copy()
    font_scale = 0.6 * label_scale  # 🔧 폰트 크기 조정

    for cls_id in np.unique(pred_mask):
        if cls_id == 0 or cls_id >= len(class_names):
            continue

        class_mask = (pred_mask == cls_id).astype(np.uint8)
        class_name = class_names[cls_id]

        num_labels, labels = cv2.connectedComponents(class_mask)
        for label_id in range(1, num_labels):
            component_mask = (labels == label_id)
            component_pixels = component_mask.sum()
            if component_pixels < 400:
                continue  # 너무 작은 영역은 라벨 생략

            coords = np.where(component_mask)
            if len(coords[0]) == 0:
                continue

            center_y = int(np.mean(coords[0]))
            center_x = int(np.mean(coords[1]))
            h, w = result.shape[:2]

            # 🔧 텍스트 크기를 미리 계산해서 배경 그리기
            temp_img = np.zeros((50, 300, 3), dtype=np.uint8)
            text_width, text_height = draw_readable_text(temp_img, class_name, (150, 25), font_scale)

            center_x = max(text_width // 2 + 10, min(center_x, w - text_width // 2 - 10))
            center_y = max(text_height + 15, min(center_y, h - 15))

            padding = 8
            # 부드러운 그림자
            shadow_offset = 1
            cv2.rectangle(result,
                          (center_x - text_width // 2 - padding + shadow_offset,
                           center_y - text_height - padding + shadow_offset),
                          (center_x + text_width // 2 + padding + shadow_offset,
                           center_y + padding + shadow_offset),
                          (0, 0, 0, 80), -1)

            # 반투명 배경
            overlay = result.copy()
            cv2.rectangle(overlay,
                          (center_x - text_width // 2 - padding, center_y - text_height - padding),
                          (center_x + text_width // 2 + padding, center_y + padding),
                          (0, 0, 0), -1)
            alpha = 0.75  # 🔧 투명도
            result = cv2.addWeighted(result, 1 - alpha, overlay, alpha, 0)

            # 클래스 색상 테두리
            if cls_id < len(class_colors_bright):
                border_color = class_colors_bright[cls_id]
            else:
                border_color = (255, 255, 255)
            cv2.rectangle(result,
                          (center_x - text_width // 2 - padding, center_y - text_height - padding),
                          (center_x + text_width // 2 + padding, center_y + padding),
                          border_color, 2)

            # 텍스트 그리기
            draw_readable_text(result, class_name, (center_x, center_y), font_scale)

    return result

# ──────────────────────────────────────────────────────────────────────────────
# 4) 밝고 선명한 오버레이 생성 (원본 + 컬러 마스크)
# ──────────────────────────────────────────────────────────────────────────────

def create_bright_overlay_rgb(original_rgb: np.ndarray, pred_mask: np.ndarray, alpha: float = 0.4) -> np.ndarray:
    """🔧 밝고 선명한 RGB 기반 오버레이 생성"""
    if original_rgb.shape[2] == 3:
        overlay_base = original_rgb.copy()
    else:
        overlay_base = cv2.cvtColor(original_rgb, cv2.COLOR_BGR2RGB)

    color_mask = mask_to_color_rgb(pred_mask)
    mask_area = (pred_mask > 0)

    if np.any(mask_area):
        overlay_base[mask_area] = (
            overlay_base[mask_area] * (1 - alpha) +
            color_mask[mask_area] * alpha
        ).astype(np.uint8)

    return overlay_base

# ──────────────────────────────────────────────────────────────────────────────
# 5) 읽기 쉬운 라벨이 적용된 예측 비교 저장
# ──────────────────────────────────────────────────────────────────────────────
from PIL import Image as PILImage

def save_prediction_comparison_readable(image_tensor, true_mask, pred_mask, save_path, original_image_path):
    """✨ 읽기 쉬운 라벨이 적용된 예측 비교 저장"""

    # 🔧 원본 이미지를 직접 로드
    if original_image_path and os.path.exists(original_image_path):
        orig_img = PILImage.open(original_image_path).convert('RGB')
        image_np = np.array(orig_img)

        # 크기 조정 (512x512로)
        if image_np.shape[:2] != (512, 512):
            image_np = cv2.resize(image_np, (512, 512))

        print(f"🔧 원본 이미지 직접 로드: {os.path.basename(original_image_path)}")

    else:
        print(f"⚠️ 원본 이미지 경로 없음, 전처리된 데이터 사용")

        # 기존 전처리된 데이터 사용 (폴백)
        if torch.is_tensor(image_tensor):
            image_np = image_tensor.permute(1, 2, 0).cpu().numpy()

            # 정규화 되었다면
            if image_np.min() >= -3 and image_np.max() <= 3:
                mean = np.array([0.485, 0.456, 0.406])
                std = np.array([0.229, 0.224, 0.225])
                image_np = image_np * std + mean
                image_np = np.clip(image_np, 0, 1)

            if image_np.max() <= 1:
                image_np = (image_np * 255).astype(np.uint8)
            else:
                image_np = image_np.astype(np.uint8)

        else:
            image_np = image_tensor

        # BGR→RGB 변환 (전처리된 데이터일 경우만)
        if len(image_np.shape) == 3 and image_np.shape[2] == 3:
            image_np = cv2.cvtColor(image_np, cv2.COLOR_BGR2RGB)
            image_np = np.ascontiguousarray(image_np, dtype=np.uint8)

    # 마스크 리사이즈
    target_h, target_w = image_np.shape[:2]
    if true_mask.shape != (target_h, target_w):
        true_mask = cv2.resize(true_mask.astype(np.uint8), (target_w, target_h), interpolation=cv2.INTER_NEAREST)
    if pred_mask.shape != (target_h, target_w):
        pred_mask = cv2.resize(pred_mask.astype(np.uint8), (target_w, target_h), interpolation=cv2.INTER_NEAREST)

    # 🔧 GT / Pred 마스크 컬러화 및 라벨 추가
    true_color_rgb = mask_to_color_rgb(true_mask)
    pred_color_rgb = mask_to_color_rgb(pred_mask)

    # 🔧 "세그먼트 중심"에 읽기 쉬운 라벨 붙이기
    true_with_labels = add_readable_center_labels(true_color_rgb.copy(), true_mask, class_names, label_scale=0.7)
    pred_with_labels = add_readable_center_labels(pred_color_rgb.copy(), pred_mask, class_names, label_scale=0.7)

    # 🔧 Overlay는 원본 이미지 + 컬러 마스크 + 라벨
    bright_overlay = create_bright_overlay_rgb(image_np, pred_mask, alpha=0.4)
    overlay_with_labels = add_readable_center_labels(bright_overlay, pred_mask, class_names, label_scale=0.6)

    # 5) 네 패널로 시각화 & 저장
    fig, axes = plt.subplots(1, 4, figsize=(20, 5))

    # 원본 이미지 (진짜 원본 색상) ✅
    axes[0].imshow(image_np)
    axes[0].set_title("Original Image", fontsize=12, fontweight='bold')
    axes[0].axis('off')

    # GT (컬러 + 라벨) ✅
    axes[1].imshow(true_with_labels)
    axes[1].set_title("Ground Truth", fontsize=12, fontweight='bold')
    axes[1].axis('off')

    # Prediction (컬러 + 라벨) ✅
    axes[2].imshow(pred_with_labels)
    axes[2].set_title("Prediction", fontsize=12, fontweight='bold')
    axes[2].axis('off')

    # Overlay (원본 색상 + 컬러 마스크 + 라벨) ✅
    axes[3].imshow(overlay_with_labels)
    axes[3].set_title("Overlay", fontsize=12, fontweight='bold')
    axes[3].axis('off')

    plt.tight_layout()
    plt.savefig(save_path, dpi=150, bbox_inches='tight')
    plt.close()

# ──────────────────────────────────────────────────────────────────────────────
# 6) 학습 히스토리 그래프 저장
# ──────────────────────────────────────────────────────────────────────────────
def plot_training_history(history, save_path="training_history.png"):
    """학습 히스토리 시각화"""
    plt.figure(figsize=(15, 5))

    # Loss 그래프
    plt.subplot(1, 3, 1)
    plt.plot(history['train_loss'], label='Train Loss', color='blue')
    plt.plot(history['val_loss'], label='Val Loss', color='red')
    plt.title('Training & Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True)

    # Dice Score 그래프
    plt.subplot(1, 3, 2)
    plt.plot(history['dice_scores'], label='Dice Score', color='green')
    plt.plot(history['iou_scores'], label='IoU Score', color='orange')
    plt.title('Segmentation Metrics')
    plt.xlabel('Epoch')
    plt.ylabel('Score')
    plt.legend()
    plt.grid(True)

    # Learning Rate 그래프 (선택 사항)
    if 'learning_rates' in history:
        plt.subplot(1, 3, 3)
        plt.plot(history['learning_rates'], label='Learning Rate', color='purple')
        plt.title('Learning Rate')
        plt.xlabel('Epoch')
        plt.ylabel('LR')
        plt.legend()
        plt.grid(True)
        plt.yscale('log')

    plt.tight_layout()
    plt.savefig(save_path, dpi=150, bbox_inches='tight')
    plt.close()


# ──────────────────────────────────────────────────────────────────────────────
# 7) 종합 리포트 생성
# ──────────────────────────────────────────────────────────────────────────────
def create_comprehensive_report(history, final_dice, final_iou, save_path="improved_report.txt"):
    """🔧 개선된 종합 성능 리포트 생성"""
    with open(save_path, 'w', encoding='utf-8') as f:
        f.write("개선된 Recycle Segmentation 학습 결과 리포트\n")
        f.write("=" * 60 + "\n\n")

        f.write("학습 설정:\n")
        f.write(f"  - 총 에포크: {len(history['train_loss'])}\n")
        f.write(f"  - 최종 Train Loss: {history['train_loss'][-1]:.4f}\n")
        f.write(f"  - 최종 Val Loss: {history['val_loss'][-1]:.4f}\n")
        f.write(f"  - 최고 Dice Score (학습 중): {max(history['dice_scores']):.4f}\n")
        f.write(f"  - 최고 IoU Score (학습 중): {max(history['iou_scores']):.4f}\n\n")

        f.write("최종 성능 평가 (개선된 예측 방식):\n")
        f.write(f"  - Dice Score: {final_dice:.4f}\n")
        f.write(f"  - IoU Score: {final_iou:.4f}\n\n")

        f.write("성능 평가:\n")
        if final_dice > 0.85:
            f.write("  ✅ 우수한 성능: 실제 배포 가능 수준\n")
        elif final_dice > 0.7:
            f.write("  ⚠️ 양호한 성능: 일부 개선 필요\n")
        elif final_dice > 0.5:
            f.write("  🔄 개선됨: 추가 튜닝으로 더 향상 가능\n")
        else:
            f.write("  ❌ 성능 부족: 추가 개선 필요\n")


In [8]:
# ===============================================================================
# 🗂️ 깔끔한 결과 저장 시스템 (4가지만)
# ===============================================================================

import os
import shutil
import matplotlib.pyplot as plt
import numpy as np
import torch
from tqdm import tqdm
from PIL import Image
import cv2

def clean_save_all_results(model, val_loader, processor, history, final_dice, final_iou, 
                          best_model_path, save_base_dir="C:/Users/USER/Desktop/Reco/results"):
    """
    🎯 깔끔하게 4가지만 저장:
    1. best_model/ - 학습된 모델
    2. visualizations/ - 모든 예측 결과 시각화 (원본+GT+예측+오버레이)
    3. performance/ - 성능 점수 및 그래프
    """
    print("🗂️ 깔끔한 결과 저장 시작...")
    
    # 기존 결과 폴더가 아닌 하위 폴더들만 정리
    viz_dir = os.path.join(save_base_dir, "visualizations")
    perf_dir = os.path.join(save_base_dir, "performance")
    
    # 기존 시각화/성능 폴더만 삭제 (best_model은 보존)
    if os.path.exists(viz_dir):
        shutil.rmtree(viz_dir)
    if os.path.exists(perf_dir):
        shutil.rmtree(perf_dir)
    
    os.makedirs(save_base_dir, exist_ok=True)
    
    # ===== 1. Best Model 확인 =====
    model_save_dir = os.path.join(save_base_dir, "best_model")
    if os.path.exists(model_save_dir):
        # 🔍 실제 파일 확인
        files = os.listdir(model_save_dir)
        if files:
            print(f"✅ 1. Best Model 확인됨: {model_save_dir}")
            print(f"   📄 파일들: {files}")
        else:
            print(f"⚠️ 1. Best Model 폴더는 있지만 비어있음")
    else:
        print(f"❌ 1. Best Model 폴더 없음: {model_save_dir}")
        # 현재 모델로 저장
        os.makedirs(model_save_dir, exist_ok=True)
        model.save_pretrained(model_save_dir)
        processor.save_pretrained(model_save_dir)
        print(f"✅ 1. Current Model 저장: {model_save_dir}")
    
    # ===== 2. 모든 예측 결과 시각화 저장 =====
    os.makedirs(viz_dir, exist_ok=True)
    print("🎨 2. 모든 예측 결과 시각화 저장 중...")
    total_saved = save_all_prediction_visualizations(model, val_loader, viz_dir)
    print(f"✅ 2. 시각화 완료: {total_saved}개 이미지 → {viz_dir}")
    
    # ===== 3. 성능 결과 저장 =====
    os.makedirs(perf_dir, exist_ok=True)
    
    # 3-1. 학습 그래프
    save_training_graphs(history, os.path.join(perf_dir, "training_history.png"))
    
    # 3-2. 성능 점수 리포트
    save_performance_report(history, final_dice, final_iou, os.path.join(perf_dir, "performance_report.txt"))
    
    print(f"✅ 3. 성능 결과 저장: {perf_dir}")
    
    print(f"\n🎉 깔끔한 저장 완료!")
    print(f"📁 저장 위치: {save_base_dir}")
    print(f"📊 구조:")
    print(f"  ├── best_model/        (학습된 모델)")
    print(f"  ├── visualizations/    ({total_saved}개 예측 시각화)")
    print(f"  └── performance/       (성능 그래프 + 리포트)")
    
    return save_base_dir

def save_all_prediction_visualizations(model, val_loader, save_dir):
    """모든 validation 데이터의 예측 결과를 4패널로 시각화하여 저장"""
    model.eval()
    total_saved = 0
    
    with torch.no_grad():
        for batch_idx, batch in enumerate(tqdm(val_loader, desc="시각화 저장")):
            imgs = batch["pixel_values"].to(device)
            labels = batch["labels"].to(device)
            filenames = batch["filename"]
            
            # 예측 수행
            _, preds = gentle_predict(batch, model, 512, len(class_names))
            
            batch_size = imgs.shape[0]
            for i in range(batch_size):
                img_path = batch["original_image_path"][i]
                img_np = np.array(Image.open(img_path).convert("RGB"))
                
                # 마스크들
                gt_mask = labels[i].cpu().numpy().astype(np.uint8)
                pred_mask = preds[i].cpu().numpy().astype(np.uint8)
                
                # 4패널 시각화 생성
                filename = f"prediction_{batch_idx:03d}_{i:02d}_{os.path.splitext(filenames[i])[0]}.png"
                save_path = os.path.join(save_dir, filename)
                
                create_4panel_visualization(img_np, gt_mask, pred_mask, save_path)
                total_saved += 1
    
    return total_saved

def create_4panel_visualization(img_np, gt_mask, pred_mask, save_path):
    """4패널 시각화: 원본 + GT + 예측 + 오버레이 (라벨 포함)"""
    fig, axes = plt.subplots(1, 4, figsize=(20, 5))
    
    # 1. 원본 이미지
    axes[0].imshow(img_np)  # 🔍 원본 이미지 그대로 표시
    axes[0].set_title("Original Image", fontsize=14, fontweight='bold')
    axes[0].axis('off')

    # 2. Ground Truth (컬러 + 라벨)
    gt_color = mask_to_color_rgb(gt_mask)
    gt_with_labels = add_readable_center_labels(gt_color.copy(), gt_mask, class_names, label_scale=0.7)  # 🔧 원래 함수 사용
    axes[1].imshow(gt_with_labels)
    axes[1].set_title("Ground Truth", fontsize=14, fontweight='bold')
    axes[1].axis('off')
    
    # 3. Prediction (컬러 + 라벨)
    pred_color = mask_to_color_rgb(pred_mask)
    pred_with_labels = add_readable_center_labels(pred_color.copy(), pred_mask, class_names, label_scale=0.7)  # 🔧 원래 함수 사용
    axes[2].imshow(pred_with_labels)
    axes[2].set_title("Prediction", fontsize=14, fontweight='bold')
    axes[2].axis('off')
    
    # 4. Overlay (원본 + 예측 + 라벨)
    overlay = create_overlay(img_np, pred_mask, alpha=0.4)
    overlay_with_labels = add_readable_center_labels(overlay, pred_mask, class_names, label_scale=0.6)  # 🔧 원래 함수 사용
    axes[3].imshow(overlay_with_labels)
    axes[3].set_title("Overlay", fontsize=14, fontweight='bold')
    axes[3].axis('off')
    
    plt.tight_layout()
    plt.savefig(save_path, dpi=150, bbox_inches='tight', facecolor='white')
    plt.close()

def create_overlay(img_np, pred_mask, alpha=0.4):
    """원본 이미지 + 예측 마스크 오버레이"""
    overlay = img_np.copy().astype(np.float32)
    color_mask = mask_to_color_rgb(pred_mask).astype(np.float32)
    
    # 마스크가 있는 영역만 오버레이
    mask_area = (pred_mask > 0)
    overlay[mask_area] = (
        overlay[mask_area] * (1 - alpha) + 
        color_mask[mask_area] * alpha
    )
    
    return overlay.astype(np.uint8)

def save_training_graphs(history, save_path):
    """학습 히스토리 그래프 저장"""
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    
    # Loss 그래프
    axes[0, 0].plot(history['train_loss'], label='Train Loss', color='blue', linewidth=2)
    axes[0, 0].plot(history['val_loss'], label='Val Loss', color='red', linewidth=2)
    axes[0, 0].set_title('Training & Validation Loss', fontsize=14, fontweight='bold')
    axes[0, 0].set_xlabel('Epoch')
    axes[0, 0].set_ylabel('Loss')
    axes[0, 0].legend()
    axes[0, 0].grid(True, alpha=0.3)
    
    # Dice Score 그래프
    axes[0, 1].plot(history['dice_scores'], label='Dice Score', color='green', linewidth=2)
    axes[0, 1].set_title('Dice Score', fontsize=14, fontweight='bold')
    axes[0, 1].set_xlabel('Epoch')
    axes[0, 1].set_ylabel('Dice Score')
    axes[0, 1].legend()
    axes[0, 1].grid(True, alpha=0.3)
    
    # IoU Score 그래프
    axes[1, 0].plot(history['iou_scores'], label='IoU Score', color='orange', linewidth=2)
    axes[1, 0].set_title('IoU Score', fontsize=14, fontweight='bold')
    axes[1, 0].set_xlabel('Epoch')
    axes[1, 0].set_ylabel('IoU Score')
    axes[1, 0].legend()
    axes[1, 0].grid(True, alpha=0.3)
    
    # Learning Rate 그래프
    if 'learning_rates' in history:
        axes[1, 1].plot(history['learning_rates'], label='Learning Rate', color='purple', linewidth=2)
        axes[1, 1].set_title('Learning Rate', fontsize=14, fontweight='bold')
        axes[1, 1].set_xlabel('Epoch')
        axes[1, 1].set_ylabel('Learning Rate')
        axes[1, 1].legend()
        axes[1, 1].grid(True, alpha=0.3)
        axes[1, 1].set_yscale('log')
    else:
        axes[1, 1].axis('off')
    
    plt.tight_layout()
    plt.savefig(save_path, dpi=200, bbox_inches='tight', facecolor='white')
    plt.close()

def save_performance_report(history, final_dice, final_iou, save_path):
    """성능 리포트 텍스트 파일 저장"""
    with open(save_path, 'w', encoding='utf-8') as f:
        f.write("🎯 Recycle Segmentation 성능 리포트\n")
        f.write("=" * 60 + "\n\n")
        
        f.write("📊 학습 결과:\n")
        f.write(f"  • 총 에포크: {len(history['train_loss'])}\n")
        f.write(f"  • 최종 Train Loss: {history['train_loss'][-1]:.4f}\n")
        f.write(f"  • 최종 Val Loss: {history['val_loss'][-1]:.4f}\n")
        f.write(f"  • 최고 Dice Score: {max(history['dice_scores']):.4f}\n")
        f.write(f"  • 최고 IoU Score: {max(history['iou_scores']):.4f}\n\n")
        
        f.write("🎯 최종 성능:\n")
        f.write(f"  • Dice Score: {final_dice:.4f}\n")
        f.write(f"  • IoU Score: {final_iou:.4f}\n\n")
        
        f.write("📈 성능 평가:\n")
        if final_dice > 0.85:
            f.write("  ✅ 우수한 성능! 실제 배포 가능한 수준\n")
        elif final_dice > 0.7:
            f.write("  🟢 좋은 성능! 실용적으로 사용 가능\n")
        elif final_dice > 0.5:
            f.write("  🟡 보통 성능. 추가 개선으로 향상 가능\n")
        else:
            f.write("  🔴 성능 부족. 추가 튜닝 필요\n")
        
        f.write(f"\n💡 개선 제안:\n")
        if final_dice < 0.7:
            f.write("  • 더 많은 데이터 추가\n")
            f.write("  • 학습률 조정\n")
            f.write("  • 데이터 증강 강화\n")
        else:
            f.write("  • 현재 성능이 우수합니다!\n")
            f.write("  • 실제 서비스 적용 고려 가능\n")

In [9]:
def run_complete_pipeline():
    """🗂️ 깔끔한 결과만 저장하는 파이프라인"""
    print("🚀 깔끔한 Recycle Segmentation 파이프라인 시작!")
    print("="*80)

    try:
        # STEP 1: 데이터 전처리
        print("\n📊 STEP 1: 데이터 전처리 시작")
        final_data_list, pixel_counter = preprocess_datasets()

        if len(final_data_list) == 0:
            print("❌ 처리할 데이터가 없습니다!")
            return

        print(f"✅ 전처리 완료: {len(final_data_list)}개 데이터")

        # STEP 2: 클래스별 개별 증강 및 Dataset 생성
        print("\n🔧 STEP 2: 클래스별 개별 증강 시작")
        device = 'cuda' if torch.cuda.is_available() else 'cpu'
        improved_data_list = advanced_class_specific_augmentation(final_data_list)

        class_weights_tensor = calculate_improved_weights(improved_data_list).to(device)
        print(f"✅ 증강 완료: {len(improved_data_list)}개 데이터")

        # 모델 로딩
        print("\n🤖 모델 및 프로세서 로딩 중...")
        processor = AutoImageProcessor.from_pretrained("apple/deeplabv3-mobilevit-small", use_fast=True)
        model = AutoModelForSemanticSegmentation.from_pretrained(
            "apple/deeplabv3-mobilevit-small",
            num_labels=len(class_names),
            id2label=id2label,
            label2id=label2id,
            ignore_mismatched_sizes=True
        ).to(device)
        print("✅ 모델 로딩 완료")

        # Train/Val 분할 (기존 코드 그대로)
        print("\n📊 Train/Val 데이터 분할 중...")
        random.seed(42)
        orig_records = []
        for rec in improved_data_list:
            base_name = os.path.basename(rec["image"])
            if "_cls" not in base_name and "_aug" not in base_name:
                orig_records.append(rec)

        random.shuffle(orig_records)
        split_idx = int(len(orig_records) * 0.8)
        train_orig_paths = {rec["image"] for rec in orig_records[:split_idx]}
        val_orig_paths = {rec["image"] for rec in orig_records[split_idx:]}

        train_list = []
        for rec in improved_data_list:
            base_name = os.path.basename(rec["image"])
            if "_cls" in base_name:
                orig_base = base_name.split("_cls")[0]
                for ext in [".jpg", ".jpeg", ".png"]:
                    orig_path = os.path.join(image_dir, orig_base + ext)
                    if orig_path in train_orig_paths:
                        train_list.append(rec)
                        break
            else:
                if rec["image"] in train_orig_paths:
                    train_list.append(rec)

        val_list = []
        for rec in improved_data_list:
            base_name = os.path.basename(rec["image"])
            if "_cls" not in base_name and "_aug" not in base_name:
                if rec["image"] in val_orig_paths:
                    val_list.append(rec)

        print(f"📊 데이터 분할 완료: Train {len(train_list)}개, Val {len(val_list)}개")

        # 데이터셋 생성
        train_ds = create_dataset_with_validation(train_list, processor, is_train=True, input_size=512)
        val_ds = create_dataset_with_validation(val_list, processor, is_train=False, input_size=512)
        
        train_loader = DataLoader(train_ds, batch_size=16, shuffle=True, num_workers=0, pin_memory=True, drop_last=True)
        val_loader = DataLoader(val_ds, batch_size=16, shuffle=False, num_workers=0, pin_memory=True)

        print(f"✅ DataLoader 생성 완료: Train batches {len(train_loader)}, Val batches {len(val_loader)}")

        # STEP 3: 모델 학습
        print("\n🚀 STEP 3: 모델 학습 시작")
        history, best_model_path = improved_training(
            model, train_loader, val_loader, processor, class_weights_tensor,
            max_epochs=400, patience=40, device=device
        )

        # Best 모델 로딩
        if best_model_path and os.path.exists(best_model_path):
            print(f"\n📥 Best 모델 로딩 중: {best_model_path}")
            try:
                model = AutoModelForSemanticSegmentation.from_pretrained(
                    best_model_path, local_files_only=True
                ).to(device)
                processor = AutoImageProcessor.from_pretrained(best_model_path)
                print("✅ Best 모델 로딩 완료!")
            except Exception as e:
                print(f"⚠️ Best 모델 로딩 실패: {e}")

        # 최종 성능 평가
        print("\n📊 최종 성능 평가 중...")
        model.eval()
        all_preds, all_targets = [], []
        with torch.no_grad():
            for batch in val_loader:
                _, pred = gentle_predict(batch, model, 512, len(class_names))
                all_preds.append(pred.cpu().numpy())
                all_targets.append(batch["labels"].cpu().numpy())
        
        pred_flat = np.concatenate([p.flatten() for p in all_preds])
        target_flat = np.concatenate([t.flatten() for t in all_targets])
        final_dice, final_iou, _ = calculate_advanced_metrics(pred_flat, target_flat, len(class_names))

        # STEP 4: 깔끔한 결과 저장
        print("\n🗂️ STEP 4: 깔끔한 결과 저장")
        save_dir = clean_save_all_results(
            model, val_loader, processor, history, 
            final_dice, final_iou, best_model_path
        )

        print(f"\n🎉 완료! 모든 결과가 저장되었습니다:")
        print(f"📁 {save_dir}")
        print(f"📊 최종 성능: Dice {final_dice:.4f}, IoU {final_iou:.4f}")

        return save_dir

    except Exception as e:
        print(f"❌ 파이프라인 실행 중 오류: {e}")
        import traceback
        traceback.print_exc()
        return None
    
run_complete_pipeline()

🚀 깔끔한 Recycle Segmentation 파이프라인 시작!

📊 STEP 1: 데이터 전처리 시작

📊 STEP 1: datasets 폴더 데이터 전처리 시작


데이터 전처리: 100%|██████████| 587/587 [00:04<00:00, 141.08it/s]


✅ 전처리 완료: 587개 데이터

🔧 STEP 2: 클래스별 개별 증강 시작

🔧 STEP 2: 클래스별 개별 증강 (can, glass, paper만)

🎯 목표 픽셀 수: 3,639,054

glass 증강 목표: 39개
paper 증강 목표: 16개

✅ 증강 완료: 총 55개 생성

📊 클래스 가중치 (sqrt 기준):
  background: 0.050
  can: 1.311
  glass: 0.948
  paper: 1.156
  plastic: 0.879
  styrofoam: 1.050
  vinyl: 0.656
✅ 증강 완료: 642개 데이터

🤖 모델 및 프로세서 로딩 중...


Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.
c:\Users\USER\anaconda3\envs\cvl\lib\site-packages\transformers\utils\generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
c:\Users\USER\anaconda3\envs\cvl\lib\site-packages\transformers\modeling_utils.py:484: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed

✅ 모델 로딩 완료

📊 Train/Val 데이터 분할 중...
📊 데이터 분할 완료: Train 529개, Val 107개
🔍 데이터 유효성 검사 시작...
✅ 유효한 데이터: 529
❌ 무효한 데이터: 0
📊 데이터셋 유효성 검사 중...
✅ 유효한 데이터: 529/529 개
🔍 데이터 유효성 검사 시작...
✅ 유효한 데이터: 107
❌ 무효한 데이터: 0
📊 데이터셋 유효성 검사 중...
✅ 유효한 데이터: 107/107 개
✅ DataLoader 생성 완료: Train batches 33, Val batches 7

🚀 STEP 3: 모델 학습 시작


...

시각화 저장:  86%|████████▌ | 6/7 [00:51<00:07,  7.64s/it]

🔧 Mask unique values: [0 6]
🔧 Available colors: 7
🔧 Class 0: 125258 pixels → RGB(0, 0, 0)
🔧 Class 6: 136886 pixels → RGB(255, 0, 128)
🔧 Color mask shape: (512, 512, 3)
🔧 Color mask range: 0 ~ 255
🔧 Color mask 채널별: R=133.2, G=0.0, B=66.8
🔧 Mask unique values: [0 6]
🔧 Available colors: 7
🔧 Class 0: 119966 pixels → RGB(0, 0, 0)
🔧 Class 6: 142178 pixels → RGB(255, 0, 128)
🔧 Color mask shape: (512, 512, 3)
🔧 Color mask range: 0 ~ 255
🔧 Color mask 채널별: R=138.3, G=0.0, B=69.4
🔧 Mask unique values: [0 6]
🔧 Available colors: 7
🔧 Class 0: 119966 pixels → RGB(0, 0, 0)
🔧 Class 6: 142178 pixels → RGB(255, 0, 128)
🔧 Color mask shape: (512, 512, 3)
🔧 Color mask range: 0 ~ 255
🔧 Color mask 채널별: R=138.3, G=0.0, B=69.4
🔧 Mask unique values: [0 6]
🔧 Available colors: 7
🔧 Class 0: 166962 pixels → RGB(0, 0, 0)
🔧 Class 6: 95182 pixels → RGB(255, 0, 128)
🔧 Color mask shape: (512, 512, 3)
🔧 Color mask range: 0 ~ 255
🔧 Color mask 채널별: R=92.6, G=0.0, B=46.5
🔧 Mask unique values: [0 6]
🔧 Available colors: 7
🔧 Cl

시각화 저장: 100%|██████████| 7/7 [00:56<00:00,  8.09s/it]


✅ 2. 시각화 완료: 107개 이미지 → C:/Users/USER/Desktop/Reco/results\visualizations
✅ 3. 성능 결과 저장: C:/Users/USER/Desktop/Reco/results\performance

🎉 깔끔한 저장 완료!
📁 저장 위치: C:/Users/USER/Desktop/Reco/results
📊 구조:
  ├── best_model/        (학습된 모델)
  ├── visualizations/    (107개 예측 시각화)
  └── performance/       (성능 그래프 + 리포트)

🎉 완료! 모든 결과가 저장되었습니다:
📁 C:/Users/USER/Desktop/Reco/results
📊 최종 성능: Dice 0.9952, IoU 0.9905


'C:/Users/USER/Desktop/Reco/results'